In [ ]:
import dataset
import classificationModel
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import OrdinalEncoder
from torch.utils.data import random_split

In [ ]:
categories = ["chirping_birds", "crackling_fire", "hand_saw", "chainsaw", "helicopter"]
audio_path = './ESC-50/audio/'
meta_path = './ESC-50/meta/'
nothing_path = './room_sounds/'

data = pd.read_csv(meta_path + 'esc50.csv')

## Remove rows where the category does not belong to categories
data = data[data.category.isin(categories)]

re_encoder = OrdinalEncoder(dtype=np.long)
re_encoder.fit(data[["category"]])
data[["category"]] = re_encoder.transform(data[["category"]])

print(re_encoder.inverse_transform(np.array([0,1,2,3,4]).reshape(-1,1)))

In [ ]:
### Small test
myModel = classificationModel.AudioClassifier()
myModel.load_state_dict(torch.load('myModel.pt'))
device = torch.device("cpu")
myModel = myModel.to(device)
# Check that it is on Cuda
next(myModel.parameters()).device
categoryList = ['chainsaw','chirping_birds','crackling_fire','hand_saw','helicopter','nothing']
ds = dataset.SoundDS(data,audio_path,False)

# myModel.eval()

total_classif = 400
classified_correctly = 0


with torch.no_grad():
    for i in range(0,total_classif):
      
        inputs = torch.Tensor(16,1,20,10)
        labels = torch.Tensor(16)
        for j in range(1,16):
            inputs[j], labels[j] = ds[np.random.randint(0,len(ds))]

        data_to_get = np.random.randint(0,len(ds))
        data = ds[data_to_get] 
        sound = ds.getAudio(data_to_get)
        inputs[0], labels[0] = data[0], data[1]

        input_m, input_s = inputs.mean(), inputs.std()
        inputs = (inputs - input_m) / input_s

        output = myModel(inputs)
        _, prediction = torch.max(output,1)
        if prediction[0] == labels[0]:
            classified_correctly += 1
        print(prediction[0])
    
acc = classified_correctly/total_classif
print(f'Accuracy: {acc:.2f}, Total items: {total_classif}')